# Homework 18: Maximum Likelihood

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sy
import utils as utils
from fractions import Fraction

from IPython.display import display, HTML

# Inline plotting
%matplotlib inline

# Make sympy print pretty math expressions
sy.init_printing()

utils.load_custom_styles()

In [2]:
def print_classification(P, objective='max'):
    labels = np.argmin(P, axis=0)+1 if objective=='min' else np.argmax(P, axis=0)+1
    out = 'Classification Result: \n$$\n'
    for i, label in enumerate(labels):
        out += '\mathbf{{x}}_{} \\to c_{}  \\\\ \n'.format(i+1, label)
    #['x{} => c{}'.format(i+1, label) for i, label in enumerate(labels)]
    out += '$$'
    return HTML(out)

def calc_distances(X, mu_k, Sigma, k):
    """
    Computes the distances between samples in X and
    the mean mu_k.
    
    If no covariance matrix is given i.e., Sigma argument is None,
    the Euclidean distance is used. Otherwse, the squared
    Mahalanobis distance is used with the given 
    """
    if Sigma is None:
        return np.linalg.norm(X - mu_k, axis=0)

    # Compute the inverse of Sigma_k
    S_inv = np.linalg.inv(Sigma[k])
    N = X.shape[1]
    result_vec = np.zeros(N)
    for i in range(N):
        x_i = X[:,i].reshape(-1, 1)
        x_i_centered = x_i - mu_k
        # Compute the Mahalanobis distance for x_i
        result_vec[i] = np.asscalar(x_i_centered.T.dot(S_inv).dot(x_i_centered))
    return result_vec    

def calc_conditional_prob(C, X, Sigma=None, verbose=True):
    n_classes = len(C)
    D = X.shape[0]
    N_ck = []
    p_x_ck_numerator = []

    for k, c in enumerate(C):
        N_ck.append(c.shape[1])
        mu_k = np.mean(c, axis=1).reshape(-1, 1)
        distances = calc_distances(X, mu_k, Sigma, k)
        p_x_ck_numerator.append(np.exp(-distances))
    P_ck = []
    p_x_ck = []
    P_ck_x = []
    for i in range(n_classes):
        P_ck.append(N_ck[i] / np.sum(N_ck))
        p_x_ck.append(p_x_ck_numerator[i] / np.sum(p_x_ck_numerator, axis=0))
        P_ck_x.append(p_x_ck[i] * P_ck[i])
    if verbose:
        for i in range(n_classes):
            print('P(c{}): {}'.format(i+1, str(P_ck[i])))
        print('---')
        for i in range(n_classes):
            print('p(x|c{}): {}'.format(i+1, utils.format_vector(p_x_ck[i])))
        print('---')
        for i in range(n_classes):
            print('P(c{}|x): {}'.format(i+1, utils.format_vector(P_ck_x[i])))
    return P_ck_x

def calc_risk_prob(C, Lambda, X, verbose=True):
    P_ck_x = calc_conditional_prob(C, X, Sigma=None, verbose=verbose)
    
    n_classes = len(P_ck_x)
    n_actions = Lambda.shape[1]
    R_ai_x = []
    for i in range(n_actions):
        lambda_i = Lambda[i,:]
        R_ai_x.append(0.0)
        for k in range(n_classes):
            R_ai_x[i] += lambda_i[k] * P_ck_x[k]
    if verbose:
        print('---')
        for i in range(n_actions):
            print('R(a{}|x): {}'.format(i+1, utils.format_vector(R_ai_x[i])))
    return R_ai_x

---
## Exercise 3.1

<img src="figures/homework-18/exercise-3.1.png" width="600" />




















In [3]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])

Compute the prior probabilities for the classes. We can compute the prior probability for each class $c_k$ as follows:

$$
P(c_k) = \frac{N_k}{\sum_l^K N_l}
$$
where $N_k$ is the number of samples in class $c_k$

Class $c_1$ has 7 samples and $c_2$ has 5 samples. In total, we have 12 samples. Therefore, $P(c_1) = 7/12=0.58$ and $P(c_2) = 5/12=0.42$.

In [4]:
N1 = c1.shape[1]
N2 = c2.shape[1]
N_sum = N1 + N2

P_c1 = N1/N_sum
P_c2 = N2/N_sum
P_c1, P_c2

We compute the class-conditional probabilities in multiple steps:
$$
p(\mathbf{x} \mid c_k) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)  }
  {  \sum_{l=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_l \rVert_2)   }
$$

Step 1: Compute the mean vector associated with each class:

In [5]:
m1 = np.mean(c1, axis=1).reshape(-1, 1)
m1

array([[-0.85714286],
       [-0.9       ]])

In [6]:
m2 = np.mean(c2, axis=1).reshape(-1, 1)
m2

array([[1.1],
       [1. ]])

Step 2: Compute $\exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)$

In [7]:
c1_numerator = np.exp(-np.linalg.norm(X - m1, axis=0))
c1_numerator

array([0.28855859, 0.07016722, 0.40201441, 0.18136432, 0.0830477 ])

Step 3: compute $\exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)$:

In [8]:
c2_numerator = np.exp(-np.linalg.norm(X - m2, axis=0))
c2_numerator

array([0.22613867, 0.90483742, 0.09769151, 0.2822644 , 0.24836923])

Step 4: Compute the class-condition probability for class 1:

$$
p(\mathbf{x} \mid c_1) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_1 \rVert_2)  }
  {  \sum_{k=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)   }
$$

In [9]:
p_x_c1 = c1_numerator / (c1_numerator + c2_numerator)
p_x_c1

array([0.5606375 , 0.07196604, 0.804502  , 0.39118439, 0.25058376])

Step 5: Compute the class-condition probability for class 2:

$$
p(\mathbf{x} \mid c_2) = 
\frac
  { \exp( - \lVert \mathbf{x} - \mu_2 \rVert_2)  }
  {  \sum_{k=1}^{K}  \exp( - \lVert \mathbf{x} - \mu_k \rVert_2)   }
$$

In [10]:
p_x_c2 = c2_numerator / (c1_numerator + c2_numerator)
p_x_c2

array([0.4393625 , 0.92803396, 0.195498  , 0.60881561, 0.74941624])

Step 7: We can now compute conditional probability, which has the following formula (Bayes' formula):

$$
P(c_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid c_k) P(c_k)}{p(\mathbf{x})}
$$
where
$$
p(\mathbf{x}) = \sum_{k=1}^K p(\mathbf{x} \mid c_k) P(c_k)
$$

Since $p(\mathbf{x})$ is the same for when computing $P(c_k\mid \mathbf{x})$, it is not necessary to compute it when we want to use the conditional probability to classify new samples. Therefore, we omit the division:
$$
P(c_k \mid \mathbf{x}) = p(\mathbf{x} \mid c_k) P(c_k)
$$

In [11]:
P_c1_x = p_x_c1 * P_c1
P_c1_x

array([0.32703854, 0.04198019, 0.46929284, 0.2281909 , 0.14617386])

In [12]:
P_c2_x = p_x_c2 * P_c2
P_c2_x

array([0.18306771, 0.38668082, 0.0814575 , 0.25367317, 0.31225677])

Step 8: Once we have the condition probabilities, we can use Bayes' Decision Rule to classify our samples $X$:

In [13]:
# Create a matrix so we can easily compute argmin
P_ck_x = np.concatenate([[P_c1_x], 
                         [P_c2_x]])
P_ck_x

array([[0.32703854, 0.04198019, 0.46929284, 0.2281909 , 0.14617386],
       [0.18306771, 0.38668082, 0.0814575 , 0.25367317, 0.31225677]])

In [14]:
print_classification(P_ck_x, objective='max')

Using the `calc_conditional_prob` function:

In [15]:
c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
               [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
               [ 1, 0.7, 1.3,  1.0,  1]])

X = np.array([[0, 1, -1,  0.7, -0.2],
              [0, 1,  0, -0.2,  1.5]])

P_ck_x = calc_conditional_prob([c1, c2], X)

print_classification(P_ck_x, objective='max')

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [ 0.5606,  0.0720,  0.8045,  0.3912,  0.2506]
p(x|c2): [ 0.4394,  0.9280,  0.1955,  0.6088,  0.7494]
---
P(c1|x): [ 0.3270,  0.0420,  0.4693,  0.2282,  0.1462]
P(c2|x): [ 0.1831,  0.3867,  0.0815,  0.2537,  0.3123]


  ---
## Exercise 3.2

<img src="figures/homework-18/exercise-3.2.png" width="600" />




















The risk of taking action $\alpha_i$ given the observation $\mathbf{x}$ as follows:

$$
R(\alpha_i \mid \mathbf{x}) = 
 \sum_{k=1}^K \lambda(\alpha_i \mid c_k) P(c_k \mid \mathbf{x})
$$

The loss function $\lambda(\alpha_i \mid c_k)$ is given by an $A \times K$ matrix $\Lambda$, where $A$ is the number of actions and $K$ is the number of classes. Here we assume that $\lambda(\alpha_i, c_k)$ corresponds to the $i$th row and $k$th column in the matrix $\Lambda$.

In [16]:
def solve_3_2():
    c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
                   [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

    c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
                   [ 1, 0.7, 1.3,  1.0,  1]])

    X = np.array([[0, 1, -1,  0.7, -0.2],
                  [0, 1,  0, -0.2,  1.5]])
    Lambda = np.array([[0, 1],
                       [1, 0]])
    R_ai_x = calc_risk_prob([c1, c2], Lambda, X)
    return print_classification(R_ai_x, objective='min')
solve_3_2()

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [ 0.5606,  0.0720,  0.8045,  0.3912,  0.2506]
p(x|c2): [ 0.4394,  0.9280,  0.1955,  0.6088,  0.7494]
---
P(c1|x): [ 0.3270,  0.0420,  0.4693,  0.2282,  0.1462]
P(c2|x): [ 0.1831,  0.3867,  0.0815,  0.2537,  0.3123]
---
R(a1|x): [ 0.1831,  0.3867,  0.0815,  0.2537,  0.3123]
R(a2|x): [ 0.3270,  0.0420,  0.4693,  0.2282,  0.1462]


---
## Exercise 3.3

<img src="figures/homework-18/exercise-3.3.png" width="600" />







In [17]:
def solve_3_3():
    c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
                   [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

    c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
                   [ 1, 0.7, 1.3,  1.0,  1]])

    X = np.array([[0, 1, -1,  0.7, -0.2],
                  [0, 1,  0, -0.2,  1.5]])
    Lambda = np.array([[0.3, 0.8],
                       [0.7, 0.2]])
    R_ai_x = calc_risk_prob([c1, c2], Lambda, X)
    return print_classification(R_ai_x, objective='min')
solve_3_3()

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [ 0.5606,  0.0720,  0.8045,  0.3912,  0.2506]
p(x|c2): [ 0.4394,  0.9280,  0.1955,  0.6088,  0.7494]
---
P(c1|x): [ 0.3270,  0.0420,  0.4693,  0.2282,  0.1462]
P(c2|x): [ 0.1831,  0.3867,  0.0815,  0.2537,  0.3123]
---
R(a1|x): [ 0.2446,  0.3219,  0.2060,  0.2714,  0.2937]
R(a2|x): [ 0.2655,  0.1067,  0.3448,  0.2105,  0.1648]


---
## Exercise 3.3 (solution manual)

<img src="figures/homework-18/exercise-3.3-extra.png" width="600" />








In [18]:
def solve_3_3_sol_manual():
    c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
                   [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

    c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
                   [ 1, 0.7, 1.3,  1.0,  1]])

    X = np.array([[0, 1, -1,  0.7, -0.2],
                  [0, 1,  0, -0.2,  1.5]])
    Lambda = np.array([[0.4, 0.8],
                       [0.6, 0.2]])
    R_ai_x = calc_risk_prob([c1, c2], Lambda, X)
    return print_classification(R_ai_x, objective='min')
solve_3_3_sol_manual()

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [ 0.5606,  0.0720,  0.8045,  0.3912,  0.2506]
p(x|c2): [ 0.4394,  0.9280,  0.1955,  0.6088,  0.7494]
---
P(c1|x): [ 0.3270,  0.0420,  0.4693,  0.2282,  0.1462]
P(c2|x): [ 0.1831,  0.3867,  0.0815,  0.2537,  0.3123]
---
R(a1|x): [ 0.2773,  0.3261,  0.2529,  0.2942,  0.3083]
R(a2|x): [ 0.2328,  0.1025,  0.2979,  0.1876,  0.1502]


---
## Exercise 3.4

<img src="figures/homework-18/exercise-3.4a.png" width="600" />





<img src="figures/homework-18/exercise-3.4b.png" width="600" />














Equation 3.69 only holds when covariance matrix is the identity matrix. The Eclidean distance can be expressed as the Mahalanobis distance if we let $\Sigma = \mathbf{I}$:

$$
\lVert \mathbf{x} - \mathbf{m}_k  \rVert_2 = 
  \sqrt{
  (\mathbf{x} - \mathbf{m}_k)^T
  \mathbf{\Sigma}^{-1}
  (\mathbf{x} - \mathbf{m}_k)
  }
$$

This means that when the covariance matrix is the identity matrix, the Mahalanobis distance reduces to Euclidean distance.

When given another covariance matrix, we need to generalise the likelihood of $\mathbf{x}$ given $c_k$ as follows:

$$
p(\mathbf{x} \mid c_k) = 
\frac
  { \exp( - (\mathbf{x} - \mathbf{m}_k)^T
  \mathbf{\Sigma}^{-1}
  (\mathbf{x} - \mathbf{m}_k)   )  }
  {  \sum_{l=1}^{K}  \exp( - 
    (\mathbf{x} - \mathbf{m}_k)^T
    \mathbf{\Sigma}^{-1}
    (\mathbf{x} - \mathbf{m}_k)
  )   }
$$

In [19]:
def solve_3_4():
    c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
                   [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

    c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
                   [ 1, 0.7, 1.3,  1.0,  1]])

    X = np.array([[0, 1, -1,  0.7, -0.2],
                  [0, 1,  0, -0.2,  1.5]])
    sigma_c1 = np.array([[1, 0],
                         [0, 2]])
    sigma_c2 = np.array([[1, 0],
                         [0, 2]])
    P_ck_x = calc_conditional_prob([c1, c2], X, [sigma_c1, sigma_c2])
    
    return print_classification(P_ck_x, objective='max')

solve_3_4()

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [ 0.6388,  0.0053,  0.9888,  0.1431,  0.1829]
p(x|c2): [ 0.3612,  0.9947,  0.0112,  0.8569,  0.8171]
---
P(c1|x): [ 0.3727,  0.0031,  0.5768,  0.0835,  0.1067]
P(c2|x): [ 0.1505,  0.4145,  0.0046,  0.3570,  0.3405]


---
## Exercise 3.5

<img src="figures/homework-18/exercise-3.5.png" width="600" />













In [20]:
def solve_3_5():
    c1 = np.array([[-1,  0, -0.5, -1.5, -2,  0,   -1],
                   [ 0, -1, -0.5, -1.5,  0, -2, -1.3]])

    c2 = np.array([[ 1, 1.3, 0.7,  2.5,  0],
                   [ 1, 0.7, 1.3,  1.0,  1]])

    X = np.array([[0, 1, -1,  0.7, -0.2],
                  [0, 1,  0, -0.2,  1.5]])

    # Compute the actual covariance matrix of c1
    mu1 = np.mean(c1, axis=1).reshape(-1, 1)
    sigma_c1 = np.dot(c1 - mu1, (c1 - mu1).T)
    
    # Compute the actual covariance matrix of c2
    mu2 = np.mean(c2, axis=1).reshape(-1, 1)
    sigma_c2 = np.dot(c2 - mu2, (c2 - mu2).T)
    
    # Compute Sigma
    sigma = (sigma_c1+sigma_c2)/2
    
    P_ck_x = calc_conditional_prob([c1, c2], X, [sigma, sigma])
    
    return print_classification(P_ck_x, objective='max')

solve_3_5()

P(c1): 0.5833333333333334
P(c2): 0.4166666666666667
---
p(x|c1): [ 0.5955,  0.0104,  0.9121,  0.4058,  0.0238]
p(x|c2): [ 0.4045,  0.9896,  0.0879,  0.5942,  0.9762]
---
P(c1|x): [ 0.3474,  0.0060,  0.5321,  0.2367,  0.0139]
P(c2|x): [ 0.1685,  0.4124,  0.0366,  0.2476,  0.4067]
